---
title: How to submit a PwBaseWorkChain 
subtitle: 
#author:
#  - name: Miki Bonacci
    #affiliations: Executable Books; Curvenote
    #orcid: 0000-0002-7859-8394
#    email: miki.bonacci@psi.ch
license:
  code: MIT
#date: 2023/01/23
---

In the following we show how to run a calculation with the `aiida-quantumespresso` plugin using the new *atomistic* `StructureData`. 
The procedure is very similar to the [old way](https://aiida-quantumespresso.readthedocs.io/en/latest/tutorials/first_pw.html), except to the fact that now properties like magnetization and charge are provided within the structure
and not in the input parameters of the `PwCalculation`.

***For now, only magnetization is supported.***

***TO MAKE IT WORK, PLEASE USE THE FOLLOWING VERSIONS OF THE RELATED CODES:***

- aiida-core: [new_StructureData branch](https://github.com/mikibonacci/aiida-core/tree/new_StructureData)
- aiida-quantumespresso: [new_StructureData branch](https://github.com/mikibonacci/aiida-quantumespresso/tree/new_StructureData)
- aiida-pseudo: [new_StructureData branch](https://github.com/mikibonacci/aiida-pseudo/tree/new_StructureData)

In [1]:
from aiida import load_profile, orm
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain
from aiida_atomistic import StructureData, StructureDataMutable

load_profile()

Profile<uuid='1a5a8d0836814a04a238c67cc7481655' name='default'>

Once loaded the necessary modules, we start building up our `StructureDataMutable`, to be then converted into the AiiDA `StructureData` (via the `to_immutable` method):

In [2]:
mutable_structure = StructureDataMutable()
mutable_structure.set_cell([[0.0, 1.8, 1.8], [1.8, 0.0, 1.8], [1.8, 1.8, 0.0]])

mutable_structure.add_atom({
            'symbol':'Si',
            'position':[1/2, 1/2, 1/2],
            'kind_name': 'Si1'
        })
mutable_structure.add_atom({
            'symbol':'Si',
            'position':[3/4, 3/4, 3/4],
            'kind_name': 'Si1'
        })

structure = mutable_structure.to_immutable()

We then generate the `PwBaseWorkChain` builder via the `get_builder_from_protocol` method, providing as input the *pw.x* code node and the structure:

In [5]:
from aiida.engine import submit, run_get_node


builder = PwBaseWorkChain.get_builder_from_protocol(
    code=orm.load_code("pw-qe-7.2@localhost"), 
    structure=structure,
    protocol="fast",
    )

run = run_get_node(builder)

09/03/2024 09:22:58 AM <518465> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1244|PwBaseWorkChain|run_process]: launching PwCalculation<1249> iteration #1
09/03/2024 09:23:01 AM <518465> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1244|PwBaseWorkChain|results]: work chain completed after 1 iterations
09/03/2024 09:23:02 AM <518465> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1244|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


After the job is finished, you can inspect the outputs as usual:

In [6]:
run[0]

{'remote_folder': <RemoteData: uuid: 11b298fa-54b2-4929-b535-d500477f6519 (pk: 1250)>,
 'retrieved': <FolderData: uuid: 9ad6c0a4-3a54-4de3-8cbe-15ab6b57c9f3 (pk: 1251)>,
 'output_parameters': <Dict: uuid: 965e3507-6907-4e70-a9fe-9bb4f2769411 (pk: 1254)>,
 'output_trajectory': <TrajectoryData: uuid: aa4c6ff9-2d31-45f4-adfb-3da14697b60c (pk: 1253)>,
 'output_band': <BandsData: uuid: 56488f69-886e-44dd-8419-e7d6244e51c1 (pk: 1252)>}

## How to deal with magnetic configurations

The first way to prepare a magnetic calculation is to trigger the `nspin` or `noncolin` in the input parameters of the `PwCalculation` respectively for *collinear* (`nspin=2`) and *non collinear* (`noncolin=.True.`) calculations.

Of course, the magnetic moments have to be provided within the `StructureData`, so that they can be used to build the [*starting_magnetization*](https://www.quantum-espresso.org/Doc/INPUT_PW.html#idm301) variable of the `&SYSTEM` namelist.


### Collinear case 

We first load the structure using Pymatgen to parse the *mcif* file for BCC Iron:

In [7]:
from pymatgen.core import Structure

#suppose you are in the `aiida-atomistic` root folder
iron_bcc = Structure.from_file('../examples/structure/data/Fe_bcc.mcif', primitive=True)
magnetic_structure = StructureData.from_pymatgen(iron_bcc)

print(magnetic_structure.properties.magmoms)

[[0.0, 0.0, 2.5]]


In [8]:
magnetic_structure.properties.kinds

['Fe1']

We can check that our structure is collinear (trivial in this case):

In [9]:
magnetic_structure.is_collinear

True

We can create again the builder as before, this time overriding `nspin` to be equal to 2 (*collinear* calculation):

In [10]:
builder = PwBaseWorkChain.get_builder_from_protocol(
    code=orm.load_code("pw-qe-7.2@localhost"), 
    structure=magnetic_structure,
    protocol="fast",
    overrides={
        "pw":{
            "parameters":{
                "SYSTEM":
                    {"nspin": 2}
            }
        }
    }
    )

In [11]:
run = run_get_node(builder)

09/03/2024 09:23:04 AM <518465> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1261|PwBaseWorkChain|run_process]: launching PwCalculation<1266> iteration #1
09/03/2024 09:23:12 AM <518465> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1261|PwBaseWorkChain|results]: work chain completed after 1 iterations
09/03/2024 09:23:13 AM <518465> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1261|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


### The output magnetic moments

For now, the new output magnetization of a structure will not be stored in the output `StructureData` (if any). 
The reason is that the output magnetization can present some discrepancies which are actually artifacts of the simulation. E.g., symmetries maybe not entirely used (due to different kinds) and so magnetization magnitudes (and orientations) are not the same as expected. 

So, the if the user needs it, he can use a `calcfuntion` to generate a new `StructureData` with the new magnetic moments (both as list of *floats* or of len=3 *lists*):

In [12]:
trajectory = run[-1].called[-1].outputs.output_trajectory # run[1].called[-1].outputs.output_trajectory
magnetic_moments = [magmom[0] for magmom in trajectory.get_array('atomic_magnetic_moments')]
print(magnetic_moments)

[2.3254]


In [13]:
from aiida_quantumespresso.utils.magnetic import generate_structure_with_magmoms

new_structure = generate_structure_with_magmoms(magnetic_structure, magnetic_moments)
print(f"New StructureData: {new_structure}\nAttached magmoms: {new_structure.properties.magmoms}")

New StructureData: uuid: cb9a17be-c75a-443a-b618-31cf5c6f4b2f (pk: 1274)
Attached magmoms: [[0.0, 0.0, 2.3254]]


If you don't want the provenance and the new `StructureData` node to be stored, add `metadata={"store_provenance": False}` to the input of the `calcfunction`. 

### Non-collinear case 

For the non-collinear case, we just need to put `noncolin: True` in the input parameters:

In [14]:
builder = PwBaseWorkChain.get_builder_from_protocol(
    code=orm.load_code("pw-qe-7.2@localhost"), 
    structure=magnetic_structure,
    protocol="fast",
    overrides={
        "pw":{
            "parameters":{
                "SYSTEM":
                    {"noncolin": True}
            }
        }
    }
    )

In [15]:
run = run_get_node(builder)

09/03/2024 09:23:14 AM <518465> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1280|PwBaseWorkChain|run_process]: launching PwCalculation<1285> iteration #1
09/03/2024 09:23:32 AM <518465> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1280|PwBaseWorkChain|results]: work chain completed after 1 iterations
09/03/2024 09:23:32 AM <518465> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1280|PwBaseWorkChain|on_terminated]: remote folders will not be cleaned


In [16]:
run[0]

{'remote_folder': <RemoteData: uuid: 812163f6-4bd2-46f0-b7fa-69f57031a00c (pk: 1286)>,
 'retrieved': <FolderData: uuid: 2bf22387-876b-41c0-82b8-7834c43c0599 (pk: 1287)>,
 'output_parameters': <Dict: uuid: c7a11701-1a6b-46fe-bae6-f3209b3544ca (pk: 1290)>,
 'output_trajectory': <TrajectoryData: uuid: 040e6c97-2704-4122-bcc4-b133d9f8a0b5 (pk: 1289)>,
 'output_band': <BandsData: uuid: 2693caf7-e783-4ac2-b4cf-bfcd7d8b4fc5 (pk: 1288)>}

Currently, no parser for non-collinear magnetization is implemented in the `aiida-quantumespresso` plugin.